# Basic test of Coiled processing NASA earthdata using Xarray and s3fs libraries with kerhunk

In [7]:
# Access libraries
import s3fs
import requests
import earthaccess

# Parallel computing libraries
import coiled
from dask.distributed import Client, LocalCluster

# Data analysis libraries
import xarray as xr

# Visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline

## Start cluster

In [10]:
cluster = coiled.Cluster(
    n_workers=10, 
    account="podaac-science", 
    region="us-west-2", 
    #worker_memory="64 GiB",
    #name='podaac-science-c0a69b8d-e'
    ) 
client = cluster.get_client()

Output()

╭────────────────────────────────────────── Package Info ──────────────────────────────────────────╮
│               ╷                                                                                  │
│   Package     │ Note                                                                             │
│ ╶─────────────┼────────────────────────────────────────────────────────────────────────────────╴ │
│   earthaccess │ Wheel built from                                                                 │
│               │ git+https://github.com/jrbourbeau/earthaccess.git@cd260a51873d0e443f9973e0d2ab   │
│               │ a9fd7aa82680                                                                     │
│   toolz       │ Wheel built from                                                                 │
│               │ git+https://github.com/pytoolz/toolz@9fef85aa68da3eef0f720fe82bb8cffa536b9d73    │
│               ╵                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [11]:
client

<Client: 'tls://10.0.53.213:8786' processes=10 threads=40, memory=148.33 GiB>

## Locate files using `earthaccess`

In [12]:
earthaccess.login("interactive")

We are already authenticated with NASA EDL


In [13]:
results = earthaccess.search_data(
    short_name="MUR-JPL-L4-GLOB-v4.1",
    cloud_hosted=True,
    #bounding_box=bounding_box,
    #temporal=("2021-04-01", "2021-05-6"),
    count=100,
    )

Granules found: 7743


## Load data with kerchunk and xarray

In [14]:
# Consolidated metdata file
metadata = earthaccess.consolidate_metadata(
    results,
    kerchunk_options={
        "concat_dims": "time", 
        "identical_dims": ['lat', 'lon']
        }, 
    #outfile="some S3 path"
    )

In [15]:
fs = earthaccess.get_s3fs_session(daac="PODAAC")
ds = xr.open_dataset(
    "reference://",
    engine="zarr",
    chunks={},
    backend_kwargs={
        "consolidated": False,
        "storage_options": {
            "fo": metadata,
            "remote_protocol": "s3",
            "remote_options": fs.storage_options,
        }
    },
)

In [25]:
ds['analysed_sst']

<xarray.DataArray 'analysed_sst' (time: 100, lat: 17999, lon: 36000)>
dask.array<open_dataset-analysed_sst, shape=(100, 17999, 36000), dtype=float32, chunksize=(1, 1023, 2047), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
  * lon      (lon) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
  * time     (time) datetime64[ns] 2002-06-01T09:00:00 ... 2002-09-08T09:00:00
Attributes:
    comment:        "Final" version using Multi-Resolution Variational Analys...
    long_name:      analysed sea surface temperature
    source:         AMSRE-REMSS, AVHRR_Pathfinder-PFV5.2-NODC_day, AVHRR_Path...
    standard_name:  sea_surface_foundation_temperature
    units:          kelvin
    valid_max:      32767
    valid_min:      -32767

## Computations

In [26]:
# Function meant to be run with dask.
# Takes an xarray.DataArray (with dimensions lat, lon), subsets spatially using
# input 2-tuples, returns mean over the subsetted region.
def regional_spatial_mean(sstdata, latbnds, lonbnds):
    future_result = sstdata.sel(lat=slice(*latbnds), lon=slice(*lonbnds)) # subset
    future_result = future_result.mean(dim=['lat','lon'], skipna=True) # mean
    return future_result.load()

In [27]:
## Bounding boxes for two regions to perform computations over

# Sub-tropical western Atlantic region:
latbnds1 = (20, 30)
lonbnds1 = (-82, -72)

# Mid-latitude eastern Pacific region:
latbnds2 = (45, 55)
lonbnds2 = (-137, -127)

In [28]:
%%time

# Computations
region1_mean_sst = regional_spatial_mean(ds['analysed_sst'], latbnds1, lonbnds1)
region2_mean_sst = regional_spatial_mean(ds['analysed_sst'], latbnds2, lonbnds2)

KeyError: 20

In [ ]:
## Plot results

fig = plt.figure(figsize=(8, 4))
axes = plt.axes()

axes.plot(region1_mean_sst['time'], region1_mean_sst.values, label='western sub-tropical Atlantic')
axes.plot(region2_mean_sst['time'], region2_mean_sst.values, label='eastern mid-latitude Pacific')

axes.legend()
axes.set_ylabel('SST [K]', fontsize=14)

In [29]:
cluster.shutdown()